In [1]:
from theano import *

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5005)


In [2]:
import theano.tensor as T

Baby Steps -Algebra

In [3]:
import numpy

In [4]:
import theano.tensor as T

In [5]:
from theano import function

x,yを数学で言う変数として定義する．  
T.dscalarとは０次元のベクトルを表す．これはtheanoで定義したものである．

In [6]:
x = T.dscalar('x')

In [7]:
y = T.dscalar('y')

In [8]:
z = x+y

functionは関数を定義するものである．  
最初のリストは関数の引数を示す．  
第二変数はよくあるのは，出力として見たいものが入る．  
式を見る感じだと，第１変数に引数で第２変数で式の定義を入れるといいかも  

In [9]:
f = function([x,y],z)

In [10]:
f(2,3)

array(5.0)

In [11]:
numpy.allclose(f(16.3, 12.1), 28.4)

True

Adding two Matrices

In [12]:
x=T.dmatrix('x')

In [13]:
y=T.dmatrix('y')

In [14]:
z = x+y

In [15]:
f=function([x,y],z)

In [16]:
f( [[1, 2], [3, 4]], [[10, 20],[30, 40]])

array([[ 11.,  22.],
       [ 33.,  44.]])

numpyで行列を定義しても大丈夫みたい

In [17]:
f(numpy.array([[1, 2], [3, 4]]), numpy.array([[10, 20], [30, 40]]))

array([[ 11.,  22.],
       [ 33.,  44.]])

# Exercise

In [18]:
import theano

In [19]:
a = theano.tensor.vector()

In [20]:
out = a+a**10

In [21]:
f = theano.function([a],out)

In [22]:
print(f( [0, 1, 2]) )

[    0.     2.  1026.]


In [23]:
f( [3, 4, 5])

array([   59052.,  1048580.,  9765630.], dtype=float32)

# Logistic Function

In [24]:
#　変数を定義する
x = T.dmatrix('x')

In [25]:
#　計算式を定義する
s = 1/( 1+T.exp(-x))

In [26]:
#　変数と計算式を元に，関数を定義する
logistic = theano.function([x],s)

In [27]:
logistic([[0,1],[-1,-2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

# tanh

In [28]:
s2 = (1+T.tanh(x/2))/2

In [29]:
logistic2 = theano.function([x],s2)

In [30]:
logistic2([[0,1],[-1,-2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

Computing More than one Thing at the Same Time

In [31]:
a,b = T.dmatrices('a','b')

In [32]:
diff = a-b

In [33]:
abs_diff = abs(diff)

In [34]:
diff_squared=diff**2

In [35]:
f= theano.function([a,b],[diff,abs_diff, diff_squared])

In [36]:
f([[1,1],[1,1]],[[0,1],[2,3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

# Setting Default Value for an Argument

In [37]:
x,y= T.dscalars('x','y')

In [38]:
z = x+y

In [39]:
from theano import In
from theano import function

f = theano.function([x,In(y,value=1)],z)

In [40]:
f(33)

array(34.0)

In [41]:
f(33,10)

array(43.0)

In [42]:
x,y,w = T.dscalars('x','y','w')

In [43]:
z=(x+y)*w

In [44]:
f = function([x,In(y, value=1), In(w,value=2, name='w_by_name')], z)

In [45]:
f(33)

array(68.0)

In [46]:
f(33,2)

array(70.0)

In [47]:
f(33,2,1)

array(35.0)

In [48]:
f(33, w_by_name=1)

array(34.0)

In [49]:
f(33, w_by_name=1,y=0)

array(33.0)

# Using Shared Variables

In [50]:
from theano import shared

In [51]:
state = shared(0)

In [52]:
inc = T.iscalar('inc')

In [53]:
accumulator = function([inc], state, updates=[(state, state+inc)])

In [54]:
print state.get_value()

0


In [55]:
accumulator(1)

array(0)

In [56]:
print state.get_value()

1


In [57]:
accumulator(300)

array(1)

In [58]:
print state.get_value()

301


In [59]:
state.set_value(-1)

In [60]:
accumulator(3)

array(-1)

In [61]:
print state.get_value()

2


In [62]:
fn_of_state = state*2 + inc

In [63]:
foo = T.scalar(dtype=state.dtype)

In [64]:
# givensによって，incにstateの値が入る
skip_shared = function([inc, foo], fn_of_state, givens=[(state, foo)])

In [65]:
skip_shared(1, 3)

array(7)

In [66]:
print state.get_value()

2


In [67]:
import theano
import theano.tensor as T

In [68]:
state = theano.shared(0)

In [69]:
inc = T.iscalar('inc')

In [70]:
accumulator = function([inc], state, updates=[(state, state+inc)],\
                      on_unused_input='ignore')

In [71]:
accumulator(10)

array(0)

In [72]:
print state.get_value()

10


In [73]:
new_state = theano.shared(0)
new_accumulator = accumulator.copy(swap={state:new_state})
new_accumulator(100)

[array(0)]

In [74]:
print new_state.get_value()

100


In [75]:
print state.get_value()

10


In [76]:
# updateしないようにする
null_accumulator = accumulator.copy(delete_updates=True)

In [77]:
null_accumulator(9000)

[array(10)]

In [78]:
print state.get_value()

10


In [79]:
print new_state.get_value()

100


In [80]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function

In [81]:
srng = RandomStreams(seed=234)

In [82]:
rv_u=srng.uniform((2,2))

In [83]:
rv_n=srng.normal((2,2))

In [84]:
f=function([],rv_u)

In [85]:
g=function([], rv_n, no_default_updates=True) #Not updating rv_n.rng

In [86]:
nearly_zeros = function([], rv_u+rv_u-2*rv_u)

In [87]:
f_val0 = f()

In [88]:
f_val1 = f()  #different numbers from f_val0

In [89]:
g_val0 = g() # different numbers from f_val0 and f_val1
g_val1 = g() # same nubers as g_val0

# Seeding Streams

In [90]:
rng_val = rv_u.rng.get_value(borrow=True) # get the rng for rv_u

In [91]:
rng_val.seed(89234) # seeds the generator

In [92]:
rv_u.rng.set_value(rng_val, borrow=True) # Assign back seeded rng

# A Real Example: Logistic Regression

In [93]:
import numpy as np
import theano
import theano.tensor as T
rng = np.random

In [94]:
N=400 # training sample size
feats=784 # number of input variables

In [95]:
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))

In [96]:
training_steps= 10000

In [97]:
x = T.dmatrix('x')
y = T.dvector('y')

w = theano.shared(rng.randn(feats), name='w')
b = theano.shared(0. , name='b')

In [98]:
print 'Initial model:'

Initial model:


In [99]:
print w.get_value()

[ -4.87461645e-01   6.85819992e-02  -6.50705022e-01   3.99410532e-01
  -4.52694141e-01  -8.55096486e-02   2.96437077e-01  -4.60280282e-01
  -2.02087030e-01   1.24355651e-01  -5.82550202e-02  -4.37285988e-01
  -1.31855144e+00   8.95418022e-01  -3.08955162e-01   9.99972704e-02
  -9.19236872e-01  -1.25705785e+00   2.18445221e-02  -7.44478327e-02
   2.58913432e-01   2.23837644e-01   9.56940931e-01  -1.17166035e+00
   2.71596611e+00  -3.34012561e-01  -1.31680053e+00   7.66230676e-02
   7.20809491e-01   1.99135431e+00  -6.64949686e-01   1.43109512e+00
  -8.80189205e-01  -1.12509161e-01   8.49779614e-01   8.73270366e-01
   2.95841221e-01   1.21809419e-01   2.17735657e+00  -1.90767198e+00
   1.42812964e-01   7.11111878e-01  -2.67739062e-01   2.03865026e-01
  -1.22735240e-01  -2.08170943e-01  -1.41883755e+00  -1.29525344e-01
  -4.75834447e-01   1.05803149e+00  -2.34106839e-01  -2.30805962e-01
   1.31439418e+00  -2.81052569e-02   4.02417475e-01  -5.37908799e-01
   3.35786969e-01  -3.99944396e-01

In [100]:
print b.get_value()

0.0


In [101]:
p_1 = 1/(1 + T.exp(-T.dot(x, w)-b))

In [102]:
prediction = p_1 >0.5

In [103]:
xent = -y* T.log(p_1) - (1-y)*T.log(1-p_1) # cross-entropy loss function

In [104]:
cost = xent.mean()+ 0.01*(w**2).sum()

In [105]:
gw, gb = T.grad(cost, [w,b])

In [106]:
# compile
# 書いたことをまとめる
# x,yを入力データとする
# 入力データxをp_1( logistic関数 )に入る
# 関数の出力が0.5以下のものをFalse,それ以外をTrue
# ここで，Pythonの仕様だと{ 1-True }は0, { 1-False }は1となる
# x,yを使って，クロスエントロピーを計算する
# クロスエントロピーをw,bで微分し，結果をgw, gbにいれる
# アップデート方法は，w-0.1*gwで行う
train = theano.function(
inputs=[x,y],
outputs=[prediction, xent],
updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))

In [107]:
predict = theano.function(inputs=[x], outputs=prediction)

In [108]:
# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

In [109]:
print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[ -9.91290963e-02   1.60272430e-01  -4.25986867e-02  -2.28603086e-02
   8.89501764e-02  -9.10176030e-02   1.06840807e-02  -1.84461767e-01
   8.38199583e-02   2.74458036e-02  -1.88692088e-01  -2.34221563e-02
  -9.94238651e-02   7.27772213e-02   1.39167992e-01   4.71935805e-02
  -5.99841326e-02  -1.43651443e-02  -4.39327975e-02   1.11154730e-01
  -4.98802590e-02   1.54214836e-01   5.96961792e-03   4.15342094e-02
  -7.40199361e-03   3.57926979e-02   6.28919073e-02  -1.65267789e-01
  -2.43510625e-04   1.39962594e-01   2.19344461e-02   3.06439964e-02
   3.36070075e-02   1.15012858e-01   6.02068553e-02   1.82602189e-01
  -3.63357467e-02   3.91889492e-02   4.11909266e-02  -1.03336289e-01
   3.23875249e-03   6.45543961e-02   7.72029194e-03  -3.48520463e-03
  -1.28629531e-01  -6.93367066e-03  -4.53187801e-02  -1.81360492e-01
   1.26689539e-01   1.23399619e-01   5.64197711e-02   4.11912281e-02
  -1.28104804e-01   5.51373360e-02   2.02535573e-02   8.53776351e-02
   5.02045539e-02   1

# Computing Gradients

In [110]:
import numpy as np
import theano
import theano.tensor as T
from theano import pp

In [111]:
x=T.dscalar('x')

In [112]:
y = x**2

In [113]:
gy = T.grad(y, x)

In [114]:
pp(gy)

'((fill((x ** TensorConstant{2}), TensorConstant{1.0}) * TensorConstant{2}) * (x ** (TensorConstant{2} - TensorConstant{1})))'

In [115]:
f = theano.function([x], gy)

In [116]:
f(4)

array(8.0)

In [117]:
np.allclose(f(94.2), 188.4)

True

In [118]:
pp(f.maker.fgraph.outputs[0])

'(TensorConstant{2.0} * x)'

In [119]:
x = T.dmatrix('x')

$ \frac{ds(x)}{dx}= s(x)\dot(1-s(x))$

In [120]:
s = T.sum(1 / (1+T.exp(-x)))

In [121]:
# sの式をxで微分する
gs = T.grad(s,x)

In [122]:
dlogistic = function([x],gs)

In [123]:
dlogistic([[0,1],[-1,-2]])

array([[ 0.25      ,  0.19661193],
       [ 0.19661193,  0.10499359]])

In [170]:
1/(1+np.exp(-0))+1/(1+np.exp(1))

0.7689414213699951

# AutoEncoder
チュートリアルに関して，だんだんlオペレータとかニューラルネットワークの計算以外の計算についての話になってきたので，一度個々で中断する．
一度，以下のサイトを頼りにAutoEncoderを実装してみる．

URL: http://aidiary.hatenablog.com/entry/20151203/1449146680  
を参考にAutoEncoderを実装する

In [124]:
import os
import sys
import timeit

import numpy as np

import theano
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

In [125]:
class dA(object):
    """Denoising Auto-Encoder class (dA)

    A denoising autoencoders tries to reconstruct the input from a corrupted
    version of it by projecting it first in a latent space and reprojecting
    it afterwards back in the input space. Please refer to Vincent et al.,2008
    for more details. If x is the input then equation (1) computes a partially
    destroyed version of x by means of a stochastic mapping q_D. Equation (2)
    computes the projection of the input into the latent space. Equation (3)
    computes the reconstruction of the input, while equation (4) computes the
    reconstruction error.
    """
    def __init__(
        self, numpy_rng, theano_rng=None, input=None,
        n_visible=784, n_hidden=500, W=None,
        bhid=None, bvis=None):
        
        self.n_visible = n_visible # input layer's node 
        self.n_hidden = n_hidden # hidden layer's node
        
        # random parameter
        if not theano_rng:
            theano_rng = RandomStreams(numpy_rng.randint(2 ** 30))
            
        # encord layers weight
        if not W:
            initial_W = np.asarray(
                        numpy_rng.uniform(
                            low=-4 * np.sqrt(6. / (n_hidden + n_visible)),
                            high=4 * np.sqrt(6. / (n_hidden + n_visible)),
                            size=(n_visible, n_hidden)
                        ),
                            dtype=theano.config.floatX
                        )
            W = theano.shared(value=initial_W, name='W', borrow=True)
        self.W = W
        
        # encord layers bias
        if not bvis:
            bvis = theano.shared(value=np.zeros(
                    n_visible,
                    dtype=theano.config.floatX
                ),
                borrow=True
            )
        self.b_prime = bvis
        
        # decord layers bias
        if not bhid:
            bhid = theano.shared(value=np.zeros(
                    n_hidden,
                    dtype=theano.config.floatX
                ),
                name='b',
                borrow=True
            )
        self.b = bhid
        
        # decord layers weight
        self.W_prime = self.W.T
        
        self.theano_rng = theano_rng
        if input is None:
            self.x = T.dmatrix(name='input')
        else:
            self.x = input
        self.params = [self.W, self.b, self.b_prime]
        
    # adding noise data
    # size : output vector size
    def get_corrupted_input(self, input, corruption_level):
        return self.theano_rng.binomial(size=input.shape, n=1,
                                        p=1 - corruption_level,
                                        dtype=theano.config.floatX) * input
    # get encoder layers output
    def get_hidden_values(self, input):
        """ Computes the values of the hidden layer """
        return T.nnet.sigmoid(T.dot(input, self.W) + self.b)
    
    # get decoder layers output
    def get_reconstructed_input(self, hidden):
        return T.nnet.sigmoid(T.dot(hidden, self.W_prime) + self.b_prime)
    
    def get_cost_updates(self, corruption_level, learning_rate):
        tilde_x = self.get_corrupted_input(self.x, corruption_level)
        y = self.get_hidden_values(tilde_x)
        z = self.get_reconstructed_input(y)

        # cross entropy
        L = - T.sum(self.x * T.log(z) + (1 - self.x) * T.log(1 - z), axis=1)

        cost = T.mean(L)
        gparams = T.grad(cost, self.params)
        updates = [
            (param, param - learning_rate * gparam)
            for param, gparam in zip(self.params, gparams)
        ]
        return (cost, updates)

In [126]:
import numpy as np
import datetime
import os
import copy

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"
Label = ['LeftHand','RightHand','LeftLeg','Rightleg','West','Chest']

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"
StorePath = "/media/takeyama/Transfer/02_ActivityResearch/"

if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+TITLE+DATE+'graph/')
GraphPath=StorePath+TITLE+DATE+'graph/'

In [130]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt

# Generate WindowFrame Function
from mymodule import window

# timer
import time

%matplotlib inline

In [131]:
DictName=os.listdir(DictionaryDataPath)
print DictName
Sensor1 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor5 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

['Chest.npz', 'LeftHand.npz', 'LeftLeg.npz', 'RightHand.npz', 'Rightleg.npz', 'West.npz']


In [132]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

****This Code is copied from https://gist.github.com/aidiary/a9c2e9ab48de446df577****

In [139]:
#coding: utf-8
import time
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
import gzip
import cPickle

class Autoencoder(object):
    def __init__(self, numpy_rng, theano_rng=None,
                 input=None,
                 n_visible=784, n_hidden=500,
                 W=None, bhid=None, bvis=None):
        self.n_visible = n_visible
        self.n_hidden = n_hidden

        if not theano_rng:
            theano_rng = RandomStreams(numpy_rng.randint(2 ** 30))

        if not W:
            # 入力層と出力層の間の重み
            initial_W = np.asarray(
                numpy_rng.uniform(
                    low=-4 * np.sqrt(6.0 / (n_hidden + n_visible)),
                    high=4 * np.sqrt(6.0 / (n_hidden + n_visible)),
                    size=(n_visible, n_hidden)
                ),
                dtype=theano.config.floatX
            )
            W = theano.shared(value=initial_W, name='W', borrow=True)

        if not bvis:
            # 入力層（visible）のユニットのバイアス
            bvis = theano.shared(
                value=np.zeros(n_visible, dtype=theano.config.floatX),
                borrow=True)

        if not bhid:
            # 隠れ層（hidden）のユニットのバイアス
            bhid = theano.shared(
                value=np.zeros(n_hidden, dtype=theano.config.floatX),
                name='b',
                borrow=True)

        # パラメータ
        self.W = W
        self.b = bhid
        self.W_prime = self.W.T
        self.b_prime = bvis
        self.params = [self.W, self.b, self.b_prime]

        self.theano_rng = theano_rng

        if input is None:
            self.x = T.dmatrix(name='input')
        else:
            self.x = input

    def get_hidden_values(self, input):
        """入力層の値を隠れ層の値に変換"""
        return T.nnet.sigmoid(T.dot(input, self.W) + self.b)

    def get_reconstructed_input(self, hidden):
        """隠れ層の値を入力層の値に逆変換"""
        return T.nnet.sigmoid(T.dot(hidden, self.W_prime) + self.b_prime)

    def get_cost_updates(self, learning_rate):
        """コスト関数と更新式のシンボルを返す"""
        # 入力を変換
        y = self.get_hidden_values(self.x)

        # 変換した値を逆変換で入力に戻す
        z = self.get_reconstructed_input(y)

        # コスト関数のシンボル
        # 元の入力と再構築した入力の交差エントロピー誤差を計算
        # 入力xがミニバッチのときLはベクトルになる
        L = - T.sum(self.x * T.log(z) + (1 - self.x) * T.log(1 - z), axis=1)

        # Lはミニバッチの各サンプルの交差エントロピー誤差なので全サンプルで平均を取る
        cost = T.mean(L)
#         cost += 0.001 * abs(self.W).sum()    # L1 regularization
#         cost += 0.001 * (self.W ** 2).sum()  # L2 regularization

        # 誤差関数の微分
        gparams = T.grad(cost, self.params)

        # 更新式のシンボル
        updates = [(param, param - learning_rate * gparam)
                   for param, gparam in zip(self.params, gparams)]

        return cost, updates

    def feedforward(self):
        """入力をフィードフォワードさせて出力を計算"""
        y = self.get_hidden_values(self.x)
        z = self.get_reconstructed_input(y)
        return z

    def __getstate__(self):
        """パラメータの状態を返す"""
        return (self.W.get_value(), self.b.get_value(), self.b_prime.get_value())

    def __setstate__(self, state):
        """パラメータの状態をセット"""
        self.W.set_value(state[0])
        self.b.set_value(state[1])
        self.b_prime.set_value(state[2])

def load_data(dataset):
    """データセットをロードしてGPUの共有変数に格納"""
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()

    def shared_dataset(data_xy, borrow=True):
        data_x, data_y = data_xy

        # 共有変数には必ずfloat型で格納
        shared_x = theano.shared(np.asarray(data_x, dtype=theano.config.floatX), borrow=borrow)
        shared_y = theano.shared(np.asarray(data_y, dtype=theano.config.floatX), borrow=borrow)

        # ラベルはint型なのでキャストして返す
        return shared_x, T.cast(shared_y, 'int32')

    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y),
            (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]

    return rval

In [152]:
type(windoW[1,1])

numpy.float64

In [159]:
learning_rate = 0.01
training_epochs = 50
batch_size = 32

# 学習データのロード
#datasets = load_data('mnist.pkl.gz')
# 自己符号化器は教師なし学習なので訓練データのラベルは使わない
#train_set_x = datasets[0][0]
WindowWidth = 256
SlidingWidth = 32
BatchSize = 32
windoW = GetWindowFrame('AccX',Sensor1['AccX'],WindowWidth,SlidingWidth,WindowDataPath)
train_set_x =  theano.shared(np.asarray(windoW, dtype=theano.config.floatX))
train_set_y =  theano.shared(np.asarray(windoW, dtype=theano.config.floatX))
test_set_x =  theano.shared(np.asarray(windoW, dtype=theano.config.floatX))
test_set_y =  theano.shared(np.asarray(windoW, dtype=theano.config.floatX))
valid_set_x =  theano.shared(np.asarray(windoW, dtype=theano.config.floatX))
valid_set_y =  theano.shared(np.asarray(windoW, dtype=theano.config.floatX))

# ミニバッチ数
n_train_batches =train_set_x.get_value(borrow=True).shape[0] / batch_size

# ミニバッチのインデックスを表すシンボル
index = T.lscalar()

# ミニバッチの学習データを表すシンボル
x = T.matrix('x')

# モデル構築
rng = np.random.RandomState(123)
theano_rng = RandomStreams(rng.randint(2 ** 30))

autoencoder = Autoencoder(numpy_rng=rng,
                           theano_rng=theano_rng,
                           input=x,
                           n_visible=windoW.shape[1],
                           n_hidden=128)

# コスト関数と更新式のシンボルを取得
cost, updates = autoencoder.get_cost_updates(learning_rate=learning_rate)

# 訓練用の関数を定義
train_da = theano.function([index],
    cost,
    updates=updates,
    givens={
        x: train_set_x[index * batch_size: (index + 1) * batch_size]
    })

# モデル訓練
fp = open("cost.txt", "w")
start_time = time.clock()
for epoch in xrange(training_epochs):
    c = []
    for batch_index in xrange(n_train_batches):
        c.append(train_da(batch_index))

    print "Training epoch %d, cost %f" % (epoch, np.mean(c))
    fp.write("%d\t%f\n" % (epoch, np.mean(c)))
    fp.flush()

end_time = time.clock()
training_time = (end_time - start_time)
fp.close()

print "time: %ds" % (training_time)

# 学習したモデルの状態を保存
f = open('autoencoder.pkl', 'wb')
cPickle.dump(autoencoder.__getstate__(), f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

this data had finished making
Training epoch 0, cost 119.411598
Training epoch 1, cost 114.275352
Training epoch 2, cost 109.250595
Training epoch 3, cost 102.603790
Training epoch 4, cost 94.786995
Training epoch 5, cost 86.582153
Training epoch 6, cost 78.471092
Training epoch 7, cost 70.630486
Training epoch 8, cost 63.075340
Training epoch 9, cost 55.777588
Training epoch 10, cost 48.681469
Training epoch 11, cost 41.730724
Training epoch 12, cost 34.885315
Training epoch 13, cost 28.110092
Training epoch 14, cost 21.378361
Training epoch 15, cost 14.618525
Training epoch 16, cost 7.760386
Training epoch 17, cost 0.898961
Training epoch 18, cost -5.899752
Training epoch 19, cost -12.637409
Training epoch 20, cost -19.322866
Training epoch 21, cost -25.962275
Training epoch 22, cost -32.562721
Training epoch 23, cost -39.128551
Training epoch 24, cost -45.663860
Training epoch 25, cost -52.174335
Training epoch 26, cost -58.663105
Training epoch 27, cost -65.128090
Training epoch 28

# https://github.com/hogefugabar/deep-learning-theano  
Theanoのコードは，どう調整すればいいのかわからないし，この人のコードは少しKerasに近いクラス設計している